# Replication Project: A quantitative theory of the gender gap in wages (Erosa, Fuster & Restuccia 2016)
Solving the dynamic model of the paper is a high dimensional problem that requires a lot of computational power. However, lot of computations can be made in parallel instead of sequentially. Consequently, we can shorten the running time significantly by relying on parallel computing packages. As recommended by Fernández-Villaverde & Zarruk (2018) we use the packages "Distributed" and "SharedArrays".
In a first step we load the packages and define the number of workers used in parallel computing. Note that we load the "Interpolations" package accompanied by the *@everywhere* command to make it accessible to all processing units used in the parallel computation.

For the rest of the code *@everywhere* is used to define objects that are unalterable and all workers of parallel computation need to access. Alterable variables and matrices are created via the *SharedArray* command.

In [1]:
using Plots, Distributed, SharedArrays, Distributions, QuantEcon, Interpolations, JLD#, Interpolations
addprocs(4) #Number of workers used in parallel computing
@everywhere using Interpolations

## Input Parameters

To increase speed additionally, we exploit properties of the Julia language. We do so by running computations within functions and using unalterable variables as parameter inputs for the model. For the latter we define a new type structure that summarizes all input parameters. We also include a function that helps us to create such a parameter structure with our desired values. The default input values are for the case of non-college females.

In [2]:
@everywhere struct Parameters
    α_1::Float64
    α_2::Float64
    γ_h::Float64
    intercept::Float64
    age1::Float64
    age2::Float64
    age3::Float64
    child::Float64
    interact::Float64
    gp_ν_s::Int64
    grid_ν_s::Vector{Float64}
    gp_ν_c::Int64
    grid_ν_c::Vector{Float64}
    markov::Matrix{Float64}
    aux_ages::Array{Int64}
    aux_ν::Array{Float64}
    gp_h::Int64
    grid_h::Vector{Float64}
    gp_n::Int64
    grid_n::Vector{Float64}
    starting_age::Int64
    J::Int64
    β::Float64
    θ::Array{Float64,2}
    Δ::Float64
    μ_h::Float64
    σ_h::Float64
    μ_v_c::Float64
    hours_factor::Float64
    ρ::Float64
    σ::Float64
    γ_n::Float64

    function Parameters(α_1 = 0.351;
                        α_2 = 0.379,
                        γ_h = 0.728,
                        intercept = 0.196,
                        age1 = 3.32,
                        age2 = -0.086,
                        age3 = 0.0007,
                        child = -6.81,
                        interact = 0.133,
                        markov,
                        grid_ν_s,
                        gp_ν_s = 7,
                        μ_ν_c = 0.7,
                        gp_ν_c = 7,
                        aux_ages = [17, 20, 25, 30, 40, 50, 55, 60, 65],
                        aux_ν = [8.0, 1.12, 0.42, 0.29, 0.25, 0.24, 0.25, 0.34, 1.6],
                        min_h = 0.0001,
                        max_h = 200,
                        gp_h = 201,
                        gp_n = 5,
                        starting_age = 17,
                        β = 0.99,
                        Δ = 3.0,
                        μ_h = 2,
                        σ_h = 0.233,
                        μ_v_c = 0.7,
                        ρ = 0.76,
                        σ = 0.79,
                        γ_n = 1.0,
                        black = false)
        grid_h = collect(range(min_h, stop = max_h, length=gp_h))
        grid_n = collect(range(0, stop = gp_n-1, length = gp_n))
        J = 4*(65-starting_age)
        percentile_ν_c = 0.9 # set percentile
        max_ν_c = -log(1-percentile_ν_c)*μ_ν_c
        grid_ν_c = collect(range(0, stop = max_ν_c, length=gp_ν_c))
        if starting_age==17 ##Non-college
            if black==false
                #baseline case
                aux1=[fill(0.0269,5*4); fill(0.0265,5*4); fill(0.0265,5*4); fill(0.0090,8*4); fill(0.0,25*4)]
                aux2=[1.0 1.44 0.76 fill(0.76,gp_n-3)']
                θ = aux1*aux2
                hours_factor = 44.2/40.5 #from table 1
            else
                #race experiment
                aux1=[fill(0.0415,5*4); fill(0.0260,5*4); fill(0.0237,5*4); fill(0.0044,8*4); fill(0.0,25*4)]
                aux2=[1.0 1.62 1.167 fill(1.06,gp_n-3)']
                θ = aux1*aux2
                hours_factor = 44.2/40.5 #from table 1
            end
        elseif starting_age==20 ## College
            aux1=[fill(0.0082,5*4); fill(0.0210,5*4); fill(0.0259,5*4); fill(0.0086,5*4); fill(0.0,25*4)]
            aux2=[1.0 2.66 0.76 fill(1.27,gp_n-3)']
            θ = aux1*aux2
            hours_factor = 46.2/42.7 #from table 1
        end
        new(α_1, α_2, γ_h, intercept, age1, age2, age3, child, interact, gp_ν_s, grid_ν_s, gp_ν_c, grid_ν_c, markov, aux_ages, aux_ν, gp_h, grid_h, gp_n, grid_n, starting_age, J, β, θ, Δ, μ_h, σ_h, μ_v_c, hours_factor, ρ, σ)
    end
end

Now we allocate the the parameter collections for the college and non-college groups. We compute the discretized $\nu_s$ grid together with its markov matrix via the tauchen method, before feeding this into the function that creates the parameter collection. We do this so that we don't need to load the QuantEcon package into every worker used in parallel computing.

P_nonc denotes the parameter collection for non-college males and females. P_c denotes the one for college individuals.

In [3]:
ρ = 0.76
gp_ν_s = 7
m_non_c = QuantEcon.tauchen(gp_ν_s,ρ,0.79,0,2)
mmpp_non_c = m_non_c.p
@everywhere mp_non_c = $mmpp_non_c
expMM_non_c = exp.(collect(m_non_c.state_values))
@everywhere expM_non_c = $expMM_non_c

P_nonc = Parameters(0.351, α_2 = 0.379, γ_h = 0.728, intercept = 0.196, age1 = 3.32, age2 = -0.086, age3 = 0.0007, child = -6.81, 
                interact = 0.133, markov = mp_non_c, grid_ν_s = expM_non_c, gp_ν_s = 7,
                aux_ages = [17, 20, 25, 30, 40, 50, 55, 60, 65], aux_ν = [8.0, 1.12, 0.42, 0.29, 0.25, 0.24, 0.25, 0.34, 1.6],
                starting_age = 17, β = 0.99, Δ = 3.0, μ_h = 2, σ_h = 0.233, μ_v_c = 0.7,σ = 0.79);
P_nonc_black = Parameters(0.351, α_2 = 0.379, γ_h = 0.728, intercept = 0.196, age1 = 3.32, age2 = -0.086, age3 = 0.0007, child = -6.81, 
                interact = 0.133, markov = mp_non_c, grid_ν_s = expM_non_c, gp_ν_s = 7,
                aux_ages = [17, 20, 25, 30, 40, 50, 55, 60, 65], aux_ν = [8.0, 1.12, 0.42, 0.29, 0.25, 0.24, 0.25, 0.34, 1.6],
                starting_age = 17, β = 0.99, Δ = 3.0, μ_h = 2, σ_h = 0.233, μ_v_c = 0.7,σ = 0.79, black = true);

m_c = QuantEcon.tauchen(gp_ν_s,ρ,1.345,0,2)
mmpp_c = m_c.p
@everywhere mp_c = $mmpp_c
expMM_c = exp.(collect(m_c.state_values))
@everywhere expM_c = $expMM_c

P_c = Parameters(-0.31, α_2 = 0.457, γ_h = 0.976, intercept = -147.8, age1 = 16.09, age2 = -0.44, age3 = 0.0039, child = -12.56,
                interact = 0.239, markov = mp_c, grid_ν_s = expM_c, gp_ν_s = 7,
                aux_ages = [20, 21, 23, 25, 30, 40, 50, 60, 65], aux_ν = [35.10, 5.0, 0.35, 0.38, 0.07, 0.05, 0.05, 0.20, 0.86],
                starting_age = 20, β = 0.99, Δ = 4.15, μ_h = 2, σ_h = 0.395, μ_v_c = 4.1, σ=1.345);

## Solving for optimal policies

### Functions

In the next field we define a few functions that are needed within the functions that will solve for the optimal policies later on. The former include primarily three value functions: $W^j(h,n,\nu)$, $H^j(h,n,\nu)$, and $V^j(h,n,\nu)$, which denote the value of working, staying at home, and the value of optimally deciding between these two options, respectively. Their functional form is defined on pages 174 and 175 in the paper. These values rely, among others, on the number of working hours $l(j,n)$, the value of optimal human capital accumulation $HCA(\cdot)$, and the cost of exerting effort $c(j,h)$. Consequently, these three functions are also defined.

Optimal human capital accumulation has a closed form solution. The function returns $HCA(\cdot)$ the value of optimal effort and optimal effort itself obtained from this closed form solution. The derivation is as follows:

\begin{align}
    \max_e c(j,h)log(1-e)+e\hat{V}^j(h(1+\Delta),n,\nu)&+(1-e)\hat{V}^j(h,n,\nu) \\
    -\frac{c(j,h)}{(1-e)} +\hat{V}^j(h(1+\Delta),n,\nu) - \hat{V}^j(h,n,\nu) &= 0 \\
    (1-e)(\hat{V}^j(h(1+\Delta),n,\nu) - \hat{V}^j(h,n,\nu)) &= c(j,h) \\
    1-e &= \frac{c(j,h)}{\hat{V}^j(h(1+\Delta),n,\nu) - \hat{V}^j(h,n,\nu)} \\
    e &= 1-\frac{c(j,h)}{\hat{V}^j(h(1+\Delta),n,\nu) - \hat{V}^j(h,n,\nu)}
\end{align}

In [4]:
@everywhere function c(j,h,P::Parameters) #cost function for exerting effort
    age = P.starting_age+(j-1)/4
    return (P.α_1+age^(P.α_2))*h^(P.γ_h)
end
@everywhere function HCA(c,V_hat,V_hat_delta) #Optimal human capital accumulation (HCA)
    e = max(1-c/(V_hat_delta-V_hat),0)        #closed form solution of optimal effort
    if e>1
        error("outside of definition")
    end
    return (c*log(1-e)+e*V_hat_delta+(1-e)*V_hat, e)
end

@everywhere function l(j,n,male_hours,P::Parameters) #working hours depending on age "j" and number of children "n"
    hours = 0.9*male_hours
    if j < 40
        hours+P.child*n+P.interact*n*j
    else
        #working hours reduction due to children becomes obsolete after age 40
        hours
    end
end

@everywhere function normalize(hours)
    hours/49.6
end

@everywhere function W(j,h,n,ν,VV_hat,VV_hat_delta,ll,P::Parameters) #Value of working
    cc = c(j,h,P)
    (opt_HCA,e) = HCA(cc,VV_hat,VV_hat_delta)
    uu = h*ν
    aux_val = h*ll + (1-ll)*uu + P.γ_n*log(1+n) + opt_HCA
    return (aux_val, e)
end

@everywhere function H(j,h,n,ν,VV_hat,P::Parameters) #Value of staying at home
    uu = h*ν
    return uu + P.γ_n*log(1+n)+VV_hat
end

@everywhere function V(j,h,n,P::Parameters) #Value of optimal labor decision
    max(H(j,h,n,P::Parameters),V(j,h,n,P::Parameters))
end

Subsequently, we define the function that solves the policy functions for men. In a first bloc all the needed matrices are preallocated. We have three value functions for working, staying at home, and the optimal decision between the two. Furthermore, we have two policy functions: optimal decision of going to work (or not) and optimal exerted effort. Additionally, we have some auxiliary variables that need preallocation to perform parallel computing.

The state of a man is fully defined by the three variables: the level of human capital, the period withing the life-cycle and the stochastic value of staying at home $\nu_s$. The three variables then determine the size of the three dimensions that the policy and value functions have. Note that the paper only denotes two state variables for men: $\nu$ and the stock of human capital. However, for men $\nu$ has two dimensions as it equals $\nu=\nu_j \nu_s$. The notation of the paper therefore implicitely also includes the same three dimensions.

We use backwards induction to solve for all the combinations of human capital level and $\nu_s$ gridpoints. We start with the last period $J$, where the value is determined myopically. For all other periods, we use the value of the next period to include the dynamic component. Due to the stochastic components, the value of entering the next period is an expected value. The stochasticity has two components: First, whether the effort of human capital accumulation is successful. This only applies in the case of working. Second, the stochastic value of staying at home $\nu_s$ that follows an AR(1) process. We approximate the AR(1) process with the Tauchen method. This means that we discretize the continuous AR(1) process to a markov matrix with according gridpoints.

In [5]:
function male_policy(P::Parameters)
    #Preallocation
    #Value functions
    W_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) #Working
    H_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) #Staying at home
    V_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) #Optimal labor decision
    #policy functions
    labor_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) #labor decision
    effort_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) # effort decision
    #initialisation of remaining variables
    age = SharedArray{Float64}(1)
    ll = SharedArray{Float64}(1)
    itp_h = SharedArray{Float64}(P.gp_h,P.gp_ν_s)
    etp_h = SharedArray{Float64}(P.gp_h,P.gp_ν_s)
    knots = (P.aux_ages,)
    int_ν_j = interpolate(knots, P.aux_ν, Gridded(Linear()))
    if P.starting_age==17 ##Non-college
        wrk_hours_NLSY_nonc = [41.61602,42.52877,43.17405,43.6543,44.5026,44.78812,44.7719,45.06112,45.66928,
        45.73491,45.88558,46.09651,46.17676,46.19064,46.91763,47.36696,47.22327,47.35094,47.10267,46.68125,
        46.69777, 46.75841, 46.66045 ,46.34579]
        wrk_hours_CPS_nonc =[20.95905,28.89055,34.11965,36.50318,38.31971,40.54175,41.49739,42.29336,42.89416,
        42.94616,43.23666,43.39517,43.57509,43.54637,43.90628,43.89597,43.97261,44.12801,43.89708,43.8616, 
        43.9502,43.9412,44.02615,43.94618,43.93134,43.98629,43.84266,44.1197,43.80469,43.8161,43.86501,43.78159, 
        43.55139,43.52779,43.54522,43.51697,43.21266,42.93177,42.50974,42.39488,42.18402,42.18699,42.42397,41.7017,
        41.246, 39.19218, 38.49695, 38.1042, 35.95146]
        hours_worked = [collect(wrk_hours_NLSY_nonc[1:21]);collect(wrk_hours_CPS_nonc[25:end])]
    elseif P.starting_age==20 ## College
        wrk_hours_NLSY_c = [31.69323,30.86017,35.1141,39.10999,41.4514,43.22748,44.10023,45.18816,45.48891,46.64455, 
        47.29931,47.54831,47.4838,47.48338,47.88516,48.17429,48.33657,48.34683,48.21116,47.64887,47.07518,
        47.0506,47.47623,47.56988]
        wrk_hours_CPS_c = [44.88889,38.41176,35.14286,37.4472,39.45081,41.07805,42.3377,43.23596,44.32782,44.65733,44.96303,
        45.36479,45.55287,45.9503,46.08944,46.59121,46.55041,46.61579,46.74215,46.8897,46.6334,47.20395,47.04182,
        46.73362,46.87761,46.68553,46.40274,46.7908,46.40085, 46.51598,46.44224,46.45074,46.23458,46.4615,45.75699,45.57569,
        44.63604,45.10394,44.52839,44.11532,44.06484,43.72736,42.92652,41.62637,41.58209,39.90182,37.64368]
        hours_worked = [collect(wrk_hours_NLSY_c[1:21]);collect(wrk_hours_CPS_c[23:end])]
    end
    itp_hours = interpolate((collect(20:65),), hours_worked, Gridded(Linear())) 
    etp_hours = extrapolate(itp_hours, Line())
    #Solving
    #Last period
    age = P.starting_age+(P.J-1)/4
    ll = normalize(etp_hours(age)) #scale working hours up by exogenous factor
    for (ind_h,h) in enumerate(P.grid_h)
        for (ind_ν, ν_s) in enumerate(P.grid_ν_s)
            ν = int_ν_j(age)*ν_s
            (W_value[ind_h,P.J,ind_ν], effort_policy[ind_h, P.J,ind_ν]) = W(P.J,h,0,ν,10^(-8),10^(-6),ll,P)
            H_value[ind_h,P.J,ind_ν]                                  = H(P.J,h,0,ν,10^(-8),P)
            V_value[ind_h,P.J,ind_ν]                                  = max.(W_value[ind_h,P.J,ind_ν],H_value[ind_h,P.J,ind_ν])
            labor_policy[ind_h,P.J,ind_ν]                             = 1*(W_value[ind_h,P.J,ind_ν]>H_value[ind_h,P.J,ind_ν])
        end
    end
    #Backwards induction
    for j in P.J-1:-1:1
        #allocation of auxiliary variables and interpolations that are constant across states
        age = P.starting_age+(j-1)/4
        ll = normalize(etp_hours(age)) #scale working hours up by exogenous factor
        #println(age)
        itp_h = interpolate((P.grid_h,P.grid_ν_s),V_value[:,j+1,:],Gridded(Linear()))
        etp_h = extrapolate(itp_h, Line())
        #solving for all combinations of human capital and ν_s gridpoints
        @sync @distributed for ind in 1:(P.gp_h*P.gp_ν_s)
            #preparing all inputs needed for later calculations
            ind_ν = convert(Int, ceil(ind/P.gp_h))
            ind_h = convert(Int, floor(mod(ind-0.05, P.gp_h))+1)
            h = P.grid_h[ind_h]       #current human capital
            h_delta = h*(1+(P.Δ/100)) #future human capital if accumulation is successful
            ν_s = P.grid_ν_s[ind_ν]   #stochastic component of staying at home
            contval = 0       #initialisation of expected value of next period if HK acc. is NOT successful
            contval_delta = 0 #initialisation of expected value of next period if HK acc. IS successful
            for (ind_ν_2, ν_s_2) in enumerate(P.grid_ν_s)
                contval       = contval      + P.markov[ind_ν,ind_ν_2]*etp_h(h,ν_s_2)
                contval_delta = contval_delta+ P.markov[ind_ν,ind_ν_2]*etp_h(h_delta,ν_s_2)
            end
            ν = int_ν_j(age)*ν_s     #value of staying at home: deterministic component times stochastic component
            #calculating the values and optimal policies
            (W_value[ind_h,j,ind_ν], effort_policy[ind_h, j,ind_ν]) = W(j, h, 0, ν, P.β*contval, P.β*contval_delta, ll, P)
            H_value[ind_h,j,ind_ν] = H(j,h,0,ν, P.β*contval,P)
            V_value[ind_h,j,ind_ν] = max.(W_value[ind_h,j,ind_ν],H_value[ind_h,j,ind_ν])
            labor_policy[ind_h,j,ind_ν] = 1*(W_value[ind_h,j,ind_ν]>H_value[ind_h,j,ind_ν])
        end
    end
    return (W_value, H_value, V_value, labor_policy, effort_policy)
end

male_policy (generic function with 1 method)

The female case is an extension of the male case. The model assumes that females differ from males in the regard of their possibility of having children. Consequently, a choice variable is added to the problem, namely the decision to have an additional child (-> child_policy). This means that we include a third policy function for this decision and a fourth value, that denotes the expected value of entering a period and behaving optimally with regard to all choice variables, including the fertility decision. We denote this as B_value.

The addition of fertility decisions opens up two more dimensions in the state of a woman compared to a man. The (stochastic) value of enjoying a newborn child at home $\nu_c$ and the number of children $n$. For the former keep in mind that $\nu$ is defined of being $\nu=\nu_j(\nu_s+\nu_c)$. However, $\nu_c$ can only be enjoyed if a woman has a newborn child. Consequently, we have $\nu=\nu_j(\nu_s+\nu_c)$ if a woman has a newborn and otherwise we have $\nu=\nu_j\nu_s$, as in the male case.

The state of women is then fully defined in five dimensions (the first three are identical to the male case):
* human capital $h$
* age/period $j$
* the stochastic value of staying at home $\nu_s$
* the stochastic value of staying at home with a newborn child $\nu_c$
* the number of children $n$

In [6]:
function female_policy(P::Parameters)
    #Value functions
    W_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #Working
    H_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #Staying at home
    V_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #Optimal labor decision
    B_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #Continuation value
    #Policy functions
    labor_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #labor decision
    effort_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #effort decision
    child_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #fertility decision
    #initialisation of remaining variables
    age = SharedArray{Float64}(1)
    ll = SharedArray{Float64}(1)
    itp_h = SharedArray{Float64}(P.gp_h,P.gp_ν_s, P.gp_ν_c)
    etp_h = SharedArray{Float64}(P.gp_h,P.gp_ν_s, P.gp_ν_c)
    knots = (P.aux_ages,)
    int_ν_j = interpolate(knots, P.aux_ν, Gridded(Linear()))
    if P.starting_age==17 ##Non-college
        wrk_hours_NLSY_nonc = [41.61602,42.52877,43.17405,43.6543,44.5026,44.78812,44.7719,45.06112,45.66928,
        45.73491,45.88558,46.09651,46.17676,46.19064,46.91763,47.36696,47.22327,47.35094,47.10267,46.68125,
        46.69777, 46.75841, 46.66045 ,46.34579]
        wrk_hours_CPS_nonc =[20.95905,28.89055,34.11965,36.50318,38.31971,40.54175,41.49739,42.29336,42.89416,
        42.94616,43.23666,43.39517,43.57509,43.54637,43.90628,43.89597,43.97261,44.12801,43.89708,43.8616, 
        43.9502,43.9412,44.02615,43.94618,43.93134,43.98629,43.84266,44.1197,43.80469,43.8161,43.86501,43.78159, 
        43.55139,43.52779,43.54522,43.51697,43.21266,42.93177,42.50974,42.39488,42.18402,42.18699,42.42397,41.7017,
        41.246, 39.19218, 38.49695, 38.1042, 35.95146]
        hours_worked = [collect(wrk_hours_NLSY_nonc[1:21]);collect(wrk_hours_CPS_nonc[25:end])]
    elseif P.starting_age==20 ## College
        wrk_hours_NLSY_c = [31.69323,30.86017,35.1141,39.10999,41.4514,43.22748,44.10023,45.18816,45.48891,46.64455, 
        47.29931,47.54831,47.4838,47.48338,47.88516,48.17429,48.33657,48.34683,48.21116,47.64887,47.07518,
        47.0506,47.47623,47.56988]
        wrk_hours_CPS_c = [44.88889,38.41176,35.14286,37.4472,39.45081,41.07805,42.3377,43.23596,44.32782,44.65733,44.96303,
        45.36479,45.55287,45.9503,46.08944,46.59121,46.55041,46.61579,46.74215,46.8897,46.6334,47.20395,47.04182,
        46.73362,46.87761,46.68553,46.40274,46.7908,46.40085, 46.51598,46.44224,46.45074,46.23458,46.4615,45.75699,45.57569,
        44.63604,45.10394,44.52839,44.11532,44.06484,43.72736,42.92652,41.62637,41.58209,39.90182,37.64368]
        hours_worked = [collect(wrk_hours_NLSY_c[1:21]);collect(wrk_hours_CPS_c[23:end])]
    end
    itp_hours = interpolate((collect(20:65),), hours_worked, Gridded(Linear())) 
    etp_hours = extrapolate(itp_hours, Line())
    #Last period
    age = P.starting_age+(P.J-1)/4
    for (ind_n,n) in enumerate(P.grid_n)
        ll = normalize(l(age,n,etp_hours(age),P))
        for (ind_h,h) in enumerate(P.grid_h)
            for (ind_ν, ν_s) in enumerate(P.grid_ν_s)
                ν = int_ν_j(age)*ν_s
                (WW, ee)                                              = W(P.J,h,n,ν,10^(-8),10^(-6),ll,P)
                HH                                                    = H(P.J,h,n,ν,10^(-8),P)
                W_value[ind_h,P.J,ind_ν,:, ind_n]                     = ones(1,P.gp_ν_c)*WW
                effort_policy[ind_h, P.J,ind_ν,:, ind_n]              = ones(1,P.gp_ν_c)*ee
                H_value[ind_h,P.J,ind_ν,:, ind_n]                     = ones(1,P.gp_ν_c)*HH
                V_value[ind_h,P.J,ind_ν,:, ind_n]                     = ones(1,P.gp_ν_c)*max.(WW,HH)
                B_value[ind_h,P.J,ind_ν,:,ind_n]                      = ones(1,P.gp_ν_c)*max.(WW,HH)
                labor_policy[ind_h,P.J,ind_ν,:, ind_n]                = ones(1,P.gp_ν_c)*(WW>HH)
            end
        end
    end
    #Backwards induction until 40 years -> irrelevant child decision
    for j in P.J-1:-1:((40-P.starting_age)*4)
        age = P.starting_age+(j-1)/4
        #println(age)
        itp_h = interpolate((P.grid_h,P.grid_ν_s,P.grid_ν_c,P.grid_n),B_value[:,j+1,:,:,:],Gridded(Linear()))
        etp_h = extrapolate(itp_h, Line())
        @sync @distributed for ind in 1:(P.gp_h*P.gp_ν_s)
            ind_ν_s = convert(Int, ceil(ind/P.gp_h))
            ind_h = convert(Int, floor(mod(ind-0.05, P.gp_h))+1)
            h = P.grid_h[ind_h]
            h_delta = h*(1+(P.Δ/100))
            ν_s = P.grid_ν_s[ind_ν_s]
            ν = int_ν_j(age)*(ν_s+0)
            for (ind_n,n) in enumerate(P.grid_n)
                ll = normalize(l(age,n,etp_hours(age),P))
                contval = 0
                contval_delta = 0
                for (ind_ν_2, ν_s_2) in enumerate(P.grid_ν_s)
                    contval       = contval      + P.markov[ind_ν_s,ind_ν_2]*etp_h(h,ν_s_2,0,n)
                    contval_delta = contval_delta+ P.markov[ind_ν_s,ind_ν_2]*etp_h(h_delta,ν_s_2,0,n)
                end
                (WW, ee)                                        = W(j, h, 0, ν, P.β*contval, P.β*contval_delta, ll, P)
                HH                                              = H(j,h,0,ν, P.β*contval,P)
                W_value[ind_h,j,ind_ν_s,:, ind_n]                 = ones(1,P.gp_ν_c)*WW
                effort_policy[ind_h, j,ind_ν_s,:, ind_n]          = ones(1,P.gp_ν_c)*ee
                H_value[ind_h,j,ind_ν_s,:, ind_n]                 = ones(1,P.gp_ν_c)*HH
                V_value[ind_h,j,ind_ν_s,:, ind_n]                 = ones(1,P.gp_ν_c)*max.(WW,HH)
                B_value[ind_h,j,ind_ν_s,:,ind_n]                  = ones(1,P.gp_ν_c)*max.(WW,HH)
                labor_policy[ind_h,j,ind_ν_s,:, ind_n]            = ones(1,P.gp_ν_c)*(WW>HH)
            end
        end
    end
    #Backwards induction before 40 years -> relevant child decision
    for j in ((40-P.starting_age)*4-1):-1:1
        age = P.starting_age+(j-1)/4
        #print(age, " ")
        itp_h = interpolate((P.grid_h,P.grid_ν_s,P.grid_ν_c,P.grid_n),B_value[:,j+1,:,:,:],Gridded(Linear()))
        etp_h = extrapolate(itp_h, Line())
        @sync @distributed for ind in 1:(P.gp_h*P.gp_ν_s)
            ind_ν_s = convert(Int, ceil(ind/P.gp_h))
            ind_h = convert(Int, floor(mod(ind-0.05, P.gp_h))+1)
            h = P.grid_h[ind_h]
            h_delta = h*(1+(P.Δ/100))
            ν_s = P.grid_ν_s[ind_ν_s]
            for (ind_n,n) in enumerate(P.grid_n)
                ll = normalize(l(age,n,etp_hours(age),P))
                for (ind_ν_c, ν_c) in enumerate(P.grid_ν_c)
                    contval = 0
                    contval_delta = 0
                    for (ind_ν_2, ν_s_2) in enumerate(P.grid_ν_s)
                        contval       = contval      + P.markov[ind_ν_s,ind_ν_2]*etp_h(h,ν_s_2,ν_c,n)
                        contval_delta = contval_delta+ P.markov[ind_ν_s,ind_ν_2]*etp_h(h_delta,ν_s_2,ν_c,n)
                    end
                    ν = int_ν_j(age)*(ν_s+ν_c)
                    (W_value[ind_h,j,ind_ν_s,ind_ν_c,ind_n], effort_policy[ind_h, j,ind_ν_s,ind_ν_c, ind_n]) = W(j, h, 0, ν, P.β*contval, P.β*contval_delta, ll, P)
                    H_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n] = H(j,h,0,ν, P.β*contval,P)
                    V_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n] = max.(W_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n],H_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n])
                    labor_policy[ind_h,j,ind_ν_s,ind_ν_c, ind_n] = 1*(W_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n]>H_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n])
                end
            end
        end
        @sync @distributed for ind in 1:((P.gp_n-1)*P.gp_ν_c)
            ind_ν_c = convert(Int, ceil(ind/(P.gp_n-1)))
            ind_n = convert(Int, floor(mod(ind-0.05, (P.gp_n-1)))+1)
            child_policy[:,j,:,ind_ν_c, ind_n] = 1*(V_value[:,j,:,ind_ν_c, ind_n+1] .> V_value[:,j,:,ind_ν_c, ind_n])
            B_value[:,j,:,ind_ν_c,ind_n] = P.θ[j,ind_n]*max.(V_value[:,j,:,ind_ν_c,ind_n+1], V_value[:,j,:,1,ind_n])+(1-P.θ[j,ind_n])*V_value[:,j,:,1, ind_n]
        end
    end
    return (W_value, H_value, V_value, B_value, labor_policy, effort_policy, child_policy)
end

female_policy (generic function with 1 method)

### Solving the Policy Functions
Now we use the functions and the input parameters to solve for optimal policies of males and females from both the college and the non-college group.
#### Males

In [7]:
@time (W_value_m_c, H_value_m_c, V_value_m_c, labor_policy_m_c, effort_policy_m_c) = male_policy(P_c);
@time (W_value_m_nonc, H_value_m_nonc, V_value_m_nonc, labor_policy_m_nonc, effort_policy_m_nonc) = male_policy(P_nonc);

 12.694320 seconds (8.81 M allocations: 458.205 MiB, 2.75% gc time)
  1.723242 seconds (288.63 k allocations: 32.513 MiB, 0.48% gc time)


In [8]:
save("m_c.jld","labor_policy", labor_policy_m_c, "effort_policy", effort_policy_m_c, "P", P_c) # store
save("m_nonc.jld","labor_policy", labor_policy_m_nonc, "effort_policy", effort_policy_m_nonc, "P", P_nonc)

#### Females

In [9]:
@time (W_value_f_c, H_value_f_c, V_value_f_c, B_value_f_c, labor_policy_f_c, effort_policy_f_c, child_policy_f_c) = female_policy(P_c);
@time (W_value_f_nonc, H_value_f_nonc, V_value_f_nonc, B_value_f_nonc, labor_policy_f_nonc, effort_policy_f_nonc, child_policy_f_nonc) = female_policy(P_nonc);
@time (W_value_f_nonc_black, H_value_f_nonc_black, V_value_f_nonc_black, B_value_f_nonc_black, labor_policy_f_nonc_black, effort_policy_f_nonc_black, child_policy_f_nonc_black) = female_policy(P_nonc_black);

 20.675037 seconds (4.67 M allocations: 314.088 MiB, 0.88% gc time)
 16.585493 seconds (693.03 k allocations: 120.271 MiB, 0.21% gc time)
 15.652983 seconds (695.51 k allocations: 120.363 MiB, 0.25% gc time)


In [10]:
save("f_c.jld","labor_policy",labor_policy_f_c, "effort_policy", effort_policy_f_c, "child_policy", child_policy_f_c);
save("f_nonc.jld","labor_policy",labor_policy_f_nonc, "effort_policy", effort_policy_f_nonc, "child_policy", child_policy_f_nonc);
save("f_nonc_black.jld","labor_policy",labor_policy_f_nonc_black, "effort_policy", effort_policy_f_nonc_black, "child_policy", child_policy_f_nonc_black);

## Running the Simulations
Subsequently, we run the simulations and store all our needed variables to compute all inputs needed for the plots and tables.

### Males

In [11]:
function male_simulation(labor_policy, effort_policy,P::Parameters)
    N = 1000 #number of simulations
    h_sim = SharedArray{Float64}(P.J+1,N)
    h_sim[1,:] = exp.(max.(rand(Normal(P.μ_h, P.σ_h),N),0))
    ν_s_sim = SharedArray{Float64}(P.J+1,N)
    ν_s_sim[1,:] = ones(N)
    income_sim = SharedArray{Float64}(P.J,N)
    labor_sim = SharedArray{Int64}(P.J,N)
    effort_sim = SharedArray{Float64}(P.J,N)
    hc_draw_sim = rand(Uniform(),P.J,N) #stochastic component to determine the success of HK accumulation  省的用概率了
    innovation_sim = rand(Normal(0,P.σ),P.J,N) #innovation of AR1 process
    working_hours_sim = SharedArray{Float64}(P.J,N)
    itp_l = interpolate((P.grid_h,collect(1:P.J),P.grid_ν_s),labor_policy[:,:,:],Gridded(Constant()))
    etp_l = extrapolate(itp_l, Flat())
    itp_e = interpolate((P.grid_h,collect(1:P.J),P.grid_ν_s),effort_policy[:,:,:],Gridded(Linear()))
    etp_e = extrapolate(itp_e, Line())
    if P.starting_age==17 ##Non-college
        wrk_hours_NLSY_nonc = [41.61602, 42.52877, 43.17405, 43.6543, 44.5026, 44.78812, 44.7719, 45.06112, 
        45.66928, 45.73491, 45.88558, 46.09651, 46.17676, 46.19064, 46.91763, 47.36696, 47.22327, 47.35094,
        47.10267, 46.68125, 46.69777, 46.75841, 46.66045 ,46.34579]
        wrk_hours_CPS_nonc =[20.95905, 28.89055, 34.11965, 36.50318, 38.31971, 40.54175, 41.49739, 42.29336,
        42.89416, 42.94616, 43.23666, 43.39517, 43.57509, 43.54637, 43.90628, 43.89597, 43.97261, 44.12801, 
        43.89708, 43.8616, 43.9502, 43.9412, 44.02615, 43.94618, 43.93134, 43.98629, 43.84266, 44.1197, 
        43.80469, 43.8161, 43.86501, 43.78159, 43.55139, 43.52779, 43.54522, 43.51697, 43.21266, 42.93177, 
        42.50974, 42.39488, 42.18402, 42.18699, 42.42397, 41.7017, 41.246, 39.19218, 38.49695, 38.1042,35.95146]
        hours_worked = [collect(wrk_hours_NLSY_nonc[1:21]);collect(wrk_hours_CPS_nonc[25:end])]
    elseif P.starting_age==20 ## College
        wrk_hours_NLSY_c = [31.69323, 30.86017, 35.1141, 39.10999, 41.4514, 43.22748, 44.10023, 45.18816, 45.48891, 46.64455, 47.29931, 47.54831, 47.4838, 47.48338, 47.88516, 48.17429, 48.33657, 48.34683, 48.21116, 47.64887, 47.07518, 47.0506, 47.47623, 47.56988]
        wrk_hours_CPS_c = [44.88889, 38.41176, 35.14286, 37.4472,39.45081,41.07805,42.3377,43.23596,44.32782,44.65733,44.96303,45.36479,45.55287,45.9503,46.08944,46.59121,46.55041,46.61579,46.74215,46.8897,46.6334,47.20395,47.04182,46.73362,46.87761,46.68553,46.40274,46.7908,46.40085, 46.51598,46.44224,46.45074,46.23458,46.4615,45.75699,45.57569,44.63604,45.10394,44.52839,44.11532,44.06484,43.72736,42.92652,41.62637,41.58209,39.90182,37.64368]
        hours_worked = [collect(wrk_hours_NLSY_c[1:21]);collect(wrk_hours_CPS_c[23:end])]
    end
    itp_hours = interpolate((collect(20:65),), hours_worked, Gridded(Linear())) 
    etp_hours = extrapolate(itp_hours, Line())
    
    
    @sync @distributed for n in 1:N
        for j in 1:P.J
            age = P.starting_age+(j-1)/4
            ν_s_sim[j+1,n] = exp(P.ρ*log(ν_s_sim[j,n])+innovation_sim[j,n])
            labor = etp_l(h_sim[j,n],j,ν_s_sim[j+1,n])
            eff = etp_e(h_sim[j,n],j,ν_s_sim[j+1,n])

            ##income 和 human capital
            #income = h_sim[j,n]* l(age, 0, P)* P.hours_factor

            labor_sim[j,n] = labor
            working_hours_sim[j,n] = labor*etp_hours(age)
            
            #print(h_sim[j,n] * l(age, 0, P) * P.hours_factor,'\t')
            #income_sim[j,n] = 1
            income_sim[j,n] = h_sim[j,n]*etp_hours(age)

            effort_sim[j,n] = eff
            if eff >= hc_draw_sim[j,n]
                h_sim[j+1,n] = h_sim[j,n]*(1+P.Δ/100)
            else
                h_sim[j+1,n] = h_sim[j,n]
            end
            
        end
    end
    return (h_sim[1:P.J,:], labor_sim, working_hours_sim, income_sim)
end

male_simulation (generic function with 1 method)

In [12]:
@time (h_sim_m_nonc, labor_sim_m_nonc, working_hours_sim_m_nonc, income_sim_m_nonc) = male_simulation(labor_policy_m_nonc, effort_policy_m_nonc,P_nonc);
@time (h_sim_m_c, labor_sim_m_c, working_hours_sim_m_c, income_sim_m_c) = male_simulation(labor_policy_m_c, effort_policy_m_c,P_c);

  4.738070 seconds (2.89 M allocations: 154.960 MiB, 2.40% gc time)
  0.148763 seconds (4.57 k allocations: 8.263 MiB)


In [13]:
quantile_needed = [0.05,0.1,0.25,0.5,0.75,0.9,0.95]
wage_percentage = SharedArray{Float64}(P_nonc.J, 7)

for (i,percentage) in enumerate(quantile_needed)
    for j in 1:P_nonc.J
        wage_percentage[j,i] = quantile(income_sim_m_nonc[j,:], percentage)
    end
end
median_first_stage = quantile(income_sim_m_nonc[1,:],0.5)  # median for the first period
standarded_wage = wage_percentage./median_first_stage
stop=(40-P_nonc.starting_age)*4+1
display(plot(P_nonc.starting_age:0.25:40,standarded_wage[1:stop,:],linewidth = 2))
print(maximum(h_sim_m_nonc))
standarded_wage

ErrorException: error compiling _plot!: error compiling _display: could not load library "libGR.so"
dlopen(libGR.so.dylib, 1): image not found

In [14]:
employment_rate = []
for i = 1:P_nonc.J
    push!(employment_rate,mean(labor_sim_m_nonc[i,:])) #the employment_rate for the ith line
end

plot(employment_rate)

ErrorException: error compiling _plot!: error compiling _display: could not load library "libGR.so"
dlopen(libGR.so.dylib, 1): image not found

### Females

In [15]:
@everywhere using Distributions #necessary for workers to call on Bernoulli function

In [16]:
function female_simulation(labor_policy, effort_policy, child_policy,P::Parameters)
    N = 1000 #number of simulations
    h_sim = SharedArray{Float64}(P.J+1,N)
    h_sim[1,:] = exp.(max.(rand(Normal(P.μ_h, P.σ_h),N),0))
    ν_s_sim = SharedArray{Float64}(P.J+1,N)
    ν_s_sim[1,:] = ones(N)
    ν_c_sim = rand(Exponential(P.μ_v_c),(P.J,N))
    fertility_sim = SharedArray{Int64}(P.J,N)
    children_sim = SharedArray{Int64}(P.J+1,N)
    children_sim[1,:] = zeros(N)
    youngest_child_age = SharedArray{Float64}(P.J+1,N)
    youngest_child_age[1,:] = fill(-1000,N)
    labor_sim = SharedArray{Int64}(P.J,N)
    unemployment_duration = SharedArray{Int64}(P.J+1,N) #counts how long an unemployment spell is in quarters
    unemployment_duration[1,:] = zeros(N)
    unemployment_spell_end = SharedArray{Float64}(P.J,N) #denotes the last period of an unemployment spell
    effort_sim = SharedArray{Float64}(P.J,N)
    hc_draw_sim = rand(Uniform(),(P.J,N)) #stochastic component to determine the success of HK accumulation
    innovation_sim = rand(Normal(0,P.σ),(P.J,N)) #innovation of AR1 process
    working_hours_sim = SharedArray{Float64}(P.J,N)
    child_sim = SharedArray{Int64}(P.J+1,N)
    itp_l = interpolate((P.grid_h,collect(1:P.J),P.grid_ν_s,P.grid_ν_c,P.grid_n),labor_policy[:,:,:,:,:],Gridded(Constant()))
    etp_l = extrapolate(itp_l, Flat())
    itp_c = interpolate((P.grid_h,collect(1:P.J),P.grid_ν_s,P.grid_ν_c,P.grid_n),child_policy[:,:,:,:,:],Gridded(Constant()))
    etp_c = extrapolate(itp_c, Flat())
    itp_e = interpolate((P.grid_h,collect(1:P.J),P.grid_ν_s,P.grid_ν_c,P.grid_n),effort_policy[:,:,:,:,:],Gridded(Linear()))
    etp_e = extrapolate(itp_e, Line())
    
    if P.starting_age==17 ##Non-college
            #if P.black==false
                wrk_hours_NLSY_nonc = [41.61602, 42.52877, 43.17405, 43.6543, 44.5026, 44.78812, 44.7719, 45.06112, 45.66928, 45.73491, 45.88558, 46.09651, 46.17676, 46.19064, 46.91763, 47.36696, 47.22327, 47.35094, 47.10267, 46.68125, 46.69777, 46.75841, 46.66045 ,46.34579]
                wrk_hours_CPS_nonc =[20.95905, 28.89055, 34.11965, 36.50318, 38.31971, 40.54175, 41.49739, 42.29336, 42.89416, 42.94616, 43.23666, 43.39517, 43.57509, 43.54637, 43.90628, 43.89597, 43.97261, 44.12801, 43.89708, 43.8616, 43.9502, 43.9412, 44.02615, 43.94618, 43.93134, 43.98629, 43.84266, 44.1197, 43.80469, 43.8161, 43.86501, 43.78159, 43.55139, 43.52779, 43.54522, 43.51697, 43.21266, 42.93177, 42.50974, 42.39488, 42.18402, 42.18699, 42.42397, 41.7017, 41.246, 39.19218, 38.49695, 38.1042, 35.95146]
                hours_worked = [collect(wrk_hours_NLSY_nonc[1:21]);collect(wrk_hours_CPS_nonc[25:end])]
            #else
                #race experiment
            #    wrk_hours_NLSY_nonc_black = [38.39248, 39.53852, 41.65616, 42.72793, 42.91534, 43.90674, 42.24459, 43.32632, 42.40185, 42.97169, 43.58034, 43.52317, 43.59142, 43.70587, 44.64614, 45.41379, 46.11264, 46.20262, 45.548, 44.64057, 45.62069, 45.46609, 44.69819, 43.45657]
            #    wrk_hours_CPS_nonc_black = [22.83747, 29.41613, 34.29444, 36.3078, 37.92713, 39.64614, 38.95276, 39.50281, 40.2061, 40.23299,40.52941,40.17025,40.58729,40.77788,40.43398,40.70308,41.10214,40.34703,40.68182,40.73382,39.96592,41.11389,40.98627,41.03655,40.78833,40.89331,41.17895,41.41185,41.23407,40.51578,40.80245,40.92878,40.68591,40.89167,41.44828,40.97869,40.88929,40.7767,40.8551,39.93467,40.11656,39.61204,38.77016,38.69668,40.54819,38.71429,37.40426,34.41509, 38.91667]
            #    hours_worked = [collect(wrk_hours_NLSY_nonc_black[1:21]);collect(wrk_hours_CPS_nonc_black[25:end])]
            #end
        elseif P.starting_age==20 ## College
            wrk_hours_NLSY_c = [31.69323, 30.86017, 35.1141, 39.10999, 41.4514, 43.22748, 44.10023, 45.18816, 45.48891, 46.64455, 47.29931, 47.54831, 47.4838, 47.48338, 47.88516, 48.17429, 48.33657, 48.34683, 48.21116, 47.64887, 47.07518, 47.0506, 47.47623, 47.56988]
            wrk_hours_CPS_c = [44.88889, 38.41176, 35.14286, 37.4472,39.45081,41.07805,42.3377,43.23596,44.32782,44.65733,44.96303,45.36479,45.55287,45.9503,46.08944,46.59121,46.55041,46.61579,46.74215,46.8897,46.6334,47.20395,47.04182,46.73362,46.87761,46.68553,46.40274,46.7908,46.40085, 46.51598,46.44224,46.45074,46.23458,46.4615,45.75699,45.57569,44.63604,45.10394,44.52839,44.11532,44.06484,43.72736,42.92652,41.62637,41.58209,39.90182,37.64368]
            hours_worked = [collect(wrk_hours_NLSY_c[1:21]);collect(wrk_hours_CPS_c[23:end])]
        end
    itp_hours = interpolate((collect(20:65),), hours_worked, Gridded(Linear())) 
    etp_hours = extrapolate(itp_hours, Line())
    
    @sync @distributed for n in 1:N
        for j in 1:P.J
            age = P.starting_age+(j-1)/4
            ν_s_sim[j+1,n] = exp(P.ρ*log(ν_s_sim[j,n])+innovation_sim[j,n])
            prob = P.θ[j,min(child_sim[j,n]+1,P.gp_n)] #probability of arising fertility decision
            if rand(Bernoulli(prob)) == 1 #woman is allowed to do fertility decision
                child = etp_c(h_sim[j,n],j,ν_s_sim[j+1,n],ν_c_sim[j,n],children_sim[j,n])
                children_sim[j+1,n] = children_sim[j,n]+trunc(Int, child)
                if child == 1
                    fertility_sim[j,n] = 1
                    labor = etp_l(h_sim[j,n],j,ν_s_sim[j+1,n],ν_c_sim[j+1,n],children_sim[j+1,n])
                    eff = etp_e(h_sim[j,n],j,ν_s_sim[j+1,n],ν_c_sim[j+1,n],children_sim[j+1,n])
                    youngest_child_age[j+1,n] = 0.25
                else
                    labor = etp_l(h_sim[j,n],j,ν_s_sim[j+1,n],0,children_sim[j+1,n])
                    eff = etp_e(h_sim[j,n],j,ν_s_sim[j+1,n],0,children_sim[j+1,n])
                    youngest_child_age[j+1,n] = youngest_child_age[j,n]+0.25
                end
                labor_sim[j,n] = trunc(Int,labor)
                unemployment_duration[j+1,n] = (-1*trunc(Int,labor)+1)*(1+unemployment_duration[j,n])
                unemployment_spell_end[j+1,n] = (-1*trunc(Int,labor)+1)*min(unemployment_duration[j,n],1)
                working_hours_sim[j,n] = labor*l(age,children_sim[j+1,n],etp_hours(age),P)
                effort_sim[j,n] = eff
                if eff >= hc_draw_sim[j,n]*labor
                    h_sim[j+1,n] = h_sim[j,n]*(1+P.Δ/100)
                else
                    h_sim[j+1,n] = h_sim[j,n]
                end
            else #woman is NOT allowed to do fertility decision-> ν_c = 0
                children_sim[j+1,n] = children_sim[j,n]
                labor = etp_l(h_sim[j,n],j,ν_s_sim[j+1,n],0,children_sim[j+1,n])
                eff = etp_e(h_sim[j,n],j,ν_s_sim[j+1,n],0,children_sim[j+1,n])
                youngest_child_age[j+1,n] = youngest_child_age[j,n]+0.25
                labor_sim[j,n] = trunc(Int,labor)
                unemployment_duration[j+1,n] = (-1*trunc(Int,labor)+1)*(1+unemployment_duration[j,n])
                unemployment_spell_end[j,n] = trunc(Int,labor)*min(unemployment_duration[j,n],1)
                working_hours_sim[j,n] = labor*l(age, children_sim[j+1,n],etp_hours(age),P)
                effort_sim[j,n] = eff
                if eff >= hc_draw_sim[j,n]
                    h_sim[j+1,n] = h_sim[j,n]*(1+P.Δ/100)
                else
                    h_sim[j+1,n] = h_sim[j,n]
                end
            end
        end
    end
    return (h_sim[1:P.J,:], children_sim[2:P.J+1,:], labor_sim, working_hours_sim, fertility_sim, unemployment_duration[2:P.J+1,:], unemployment_spell_end[2:P.J,:], max.(youngest_child_age[2:P.J+1,:],0))
end

female_simulation (generic function with 1 method)

In [17]:
group = ["c","nonc","nonc_black"]
for g in group
    MyLine = Meta.parse("@time (h_sim_f_$(g), child_sim_f_$(g), labor_sim_f_$(g), working_hours_sim_f_$(g), fertility_sim_f_$(g), unemployment_duration_f_$(g), unemployment_spell_end_f_$(g), youngest_child_age_f_$(g)) = female_simulation(labor_policy_f_$(g), effort_policy_f_$(g), child_policy_f_$(g), P_$(g));")
    eval(MyLine)
end

 13.450803 seconds (1.83 M allocations: 303.684 MiB, 2.68% gc time)
  2.109977 seconds (47.68 k allocations: 232.077 MiB, 8.73% gc time)
  1.913803 seconds (7.87 k allocations: 230.018 MiB, 8.53% gc time)


## Tables and Plots

### Table 10/21

In [18]:
#average fertility second group 20-24
group = ["c", "nonc"]#,"nonc_black"]
tab_11_sim = Array{Any}(undef,5,length(group))
tab_11_sim[1,1]=0.0
Istart = (17-P_nonc.starting_age)*4+1; Iend = (17-P_nonc.starting_age)*4;
aux = sum(fertility_sim_f_nonc[Istart:Iend,:]; dims=1)
tab_11_sim[1,2] = mean(aux)
#aux = sum(fertility_sim_f_nonc_black[Istart:Iend,:]; dims=1)
#tab_11_sim[1,3] = mean(aux)

for (ind_g, g) in enumerate(group)
    ages = [17, 20, 25, 30, 35, 40]
    for i in 2:5
        MyLine = Meta.parse("Istart = (ages[$(i)]-P_$(g).starting_age)*4+1; Iend = (ages[$(i)+1]-P_$(g).starting_age)*4; aux = sum(fertility_sim_f_$(g)[Istart:Iend,:]; dims=1)")
        eval(MyLine)
        tab_11_sim[i,ind_g]=mean(aux)
    end
end
aux = ["College" "Non-College"]# "Black"]
tab_11_sim_described = [["College" "Non-College" "Black"]; tab_11_sim]


UndefVarError: UndefVarError: ages not defined

### Table 11/20

In [19]:
group = ["f_c","f_nonc","f_nonc_black"]
tab_11_sim = Array{Any}(undef,6,length(group))
for (ind_g, g) in enumerate(group)
    criteria = ["==0.25)","==0.5)", "==0.75)","==1.0)","<1.0).*(youngest_child_age_$(g).<=5.0)","<5.0).*(youngest_child_age_$(g).<=6.0)"]
    for (ind_i, i) in enumerate(criteria)
        MyLine1 = Meta.parse("binary = 1 .*(youngest_child_age_$(g) .$(i)") #assigns 1 to mothers with youngest child of age of desired criteria
        MyLine2 = Meta.parse("labor_bin = labor_sim_$(g).*binary") #assigns 1 to mothers that work and fulfill the prior criterium of youngest child age
        #println(MyLine1)
        eval(MyLine1)
        #println(MyLine2)
        eval(MyLine2)
        println(sum(binary))
        tab_11_sim[ind_i, ind_g] = sum(labor_bin)/sum(binary) #if NaN, mostly because binary=0
    end
end
tab_11_sim_described = ["College" "Non-College" "Black" ;tab_11_sim]
lengths = [" "; "1Q"; "2Q"; "3Q"; "4Q"; "1y-5y"; "5y-6y"]
tab_11_sim_described = [lengths tab_11_sim_described]

0
0
0
0
180000
180000
0
0
0
0
192000
192000
0
0
0
0
192000
192000


7×4 Array{Any,2}:
 " "         "College"     "Non-College"     "Black"
 "1Q"     NaN           NaN               NaN       
 "2Q"     NaN           NaN               NaN       
 "3Q"     NaN           NaN               NaN       
 "4Q"     NaN           NaN               NaN       
 "1y-5y"    0.834422      0.76912           0.770557
 "5y-6y"    0.834422      0.76912           0.770557

In [20]:
binary = 1 .* (youngest_child_age_f_c .== 0.25)
labor_bin = labor_sim_f_c .* binary
sum(binary)

0

### Table 13

In [21]:
period_40 = (40-P_nonc.starting_age)*4+1
total_spells=sum(unemployment_spell_end_f_nonc[1:period_40,:])
aux = unemployment_duration_f_nonc[1:period_40,:].*unemployment_spell_end_f_nonc[1:period_40,:]
criteria = ["x==1.0", ""]
for (ind_c, c) in enumerate(criteria)
    spell_1 = count(x->(x==1.0),aux) #counts numbers of spells of 1 quarter length
    spell_1/total_spells #relative amount of spells of 1 quarter length
end

### Table 12

In [22]:
#like table 13 but only takes mothers into account
period_40 = (40-P_nonc.starting_age)*4+1
total_spells=sum(unemployment_spell_end_f_nonc[1:period_40,:].*(child_sim_f_nonc[1:period_40,:].>0))

aux = unemployment_duration_f_nonc[1:period_40,:].*unemployment_spell_end_f_nonc[1:period_40,:].*(child_sim_f_nonc[1:period_40,:].>0)
spell_1 = count(x->(x==1.0),aux) #counts numbers of spells of 1 quarter length
spell_1/total_spells #relative amount of spells of 1 quarter length

NaN

# Random Stuff
### ...

In [23]:
quantile_needed = [0.05,0.1,0.25,0.5,0.75,0.9,0.95]
wage_percentage = SharedArray{Float64}(P_nonc.J, 7)

for (i,percentage) in enumerate(quantile_needed)
    for j in 1:P_nonc.J
        wage_percentage[j,i] = quantile(income_sim_m_nonc[j,:], percentage)
    end
end

In [24]:
median_first_stage = quantile(income_sim_m_nonc[1,:],0.5)  # median for the first period
standarded_wage = wage_percentage./median_first_stage

192×7 Array{Float64,2}:
 0.657846  0.735195  0.848442  1.0      1.16109  1.31939  1.44051
 0.680224  0.745662  0.869593  1.03005  1.18921  1.35892  1.48153
 0.691417  0.766817  0.890218  1.0557   1.22369  1.40204  1.5288 
 0.7101    0.779476  0.913128  1.08653  1.25503  1.43245  1.56487
 0.725673  0.795346  0.9396    1.10937  1.28491  1.46663  1.61108
 0.740308  0.812077  0.958583  1.13372  1.31414  1.50499  1.65935
 0.752278  0.835662  0.98141   1.15663  1.34212  1.54815  1.70093
 0.775554  0.856867  0.99809   1.18299  1.37568  1.58129  1.73079
 0.784093  0.866857  1.01964   1.21421  1.41311  1.62029  1.76947
 0.802878  0.889371  1.03988   1.23368  1.44972  1.66487  1.80657
 0.815764  0.902317  1.06385   1.25732  1.48135  1.70078  1.84391
 0.826884  0.918925  1.0856    1.28774  1.51391  1.74481  1.89169
 0.84085   0.935477  1.10884   1.31433  1.54401  1.78869  1.94675
 ⋮                                               ⋮               
 1.03585   1.25572   1.70199   2.45866  3.60782  5.4

In [25]:
stop=(40-P_nonc.starting_age)*4+1
plot(P_nonc.starting_age:0.25:40,standarded_wage[1:stop,:],linewidth = 2)
#plot!(standarded_wage[1:stop,2],linewidth = 2)
#plot!(standarded_wage[1:stop,3],linewidth = 2)
#plot!(standarded_wage[1:stop,4],linewidth = 2)
#plot!(standarded_wage[1:stop,5],linewidth = 2)
#plot!(standarded_wage[1:stop,6],linewidth = 2)
#plot!(standarded_wage[1:stop,7],linewidth = 2)
print(maximum(h_sim_m_nonc))

ErrorException: error compiling _plot!: error compiling _display: could not load library "libGR.so"
dlopen(libGR.so.dylib, 1): image not found

In [26]:
employment_rate = []
for i = 1:P_nonc.J
    push!(employment_rate,mean(labor_sim_m_nonc[i,:])) #the employment_rate for the ith line
end

plot(employment_rate)

ErrorException: error compiling _plot!: error compiling _display: could not load library "libGR.so"
dlopen(libGR.so.dylib, 1): image not found

In [27]:
arr = []
for i in 20:45
    push!(arr,l(i,0,P_c))
end
plot(20:45,arr, label="working hours per week", xlabel="age")

MethodError: MethodError: no method matching l(::Int64, ::Int64, ::Parameters)
Closest candidates are:
  l(::Any, ::Any, ::Any, !Matched::Parameters) at In[4]:14

In [28]:
wrk_hours_NLSY_all =[39.09459,39.56225,41.15072,42.5114,43.73147,44.39577,44.60279,45.09376,45.62251,45.96919,46.25466,46.47443,46.51523,46.52374,47.16737,47.57628,47.51178,47.61147,47.39395,46.93997,46.79821,46.8379,46.88184,46.67376]
wrk_hours_NLSY_nonc = [41.61602, 42.52877, 43.17405, 43.6543, 44.5026, 44.78812, 44.7719, 45.06112, 45.66928, 45.73491, 45.88558, 46.09651, 46.17676, 46.19064, 46.91763, 47.36696, 47.22327, 47.35094, 47.10267, 46.68125, 46.69777, 46.75841, 46.66045 ,46.34579]
wrk_hours_NLSY_nonc_black = [38.39248, 39.53852, 41.65616, 42.72793, 42.91534, 43.90674, 42.24459, 43.32632, 42.40185, 42.97169, 43.58034, 43.52317, 43.59142, 43.70587, 44.64614, 45.41379, 46.11264, 46.20262, 45.548, 44.64057, 45.62069, 45.46609, 44.69819, 43.45657]
wrk_hours_NLSY_c = [31.69323, 30.86017, 35.1141, 39.10999, 41.4514, 43.22748, 44.10023, 45.18816, 45.48891, 46.64455, 47.29931, 47.54831, 47.4838, 47.48338, 47.88516, 48.17429, 48.33657, 48.34683, 48.21116, 47.64887, 47.07518, 47.0506, 47.47623, 47.56988]
ages_NLSY = collect(20:43)
wrk_hours_CPS_all=[20.95905,28.89055,34.1324,36.50721,38.2761,40.23252,41.16984,42.06722,42.78102,43.01234,43.49506,43.71818,43.94692,44.0628,44.38766,44.49844,44.59843,44.86217,44.68788,44.69109,44.8023,44.85968,44.86664,44.96542,44.92937,44.86303,44.82444,44.95251,44.65644,44.81043,44.71614,44.71489,44.52945,44.52751,44.49105,44.53985,44.12905,43.89655,43.27912,43.3735,43.03038,42.87579,42.97711,42.38791,41.81822,40.03535,39.62034,38.7056,36.56108]
wrk_hours_CPS_c = [44.88889, 38.41176, 35.14286, 37.4472,39.45081,41.07805,42.3377,43.23596,44.32782,44.65733,44.96303,45.36479,45.55287,45.9503,46.08944,46.59121,46.55041,46.61579,46.74215,46.8897,46.6334,47.20395,47.04182,46.73362,46.87761,46.68553,46.40274,46.7908,46.40085, 46.51598,46.44224,46.45074,46.23458,46.4615,45.75699,45.57569,44.63604,45.10394,44.52839,44.11532,44.06484,43.72736,42.92652,41.62637,41.58209,39.90182,37.64368]
wrk_hours_CPS_nonc =[20.95905, 28.89055, 34.11965, 36.50318, 38.31971, 40.54175, 41.49739, 42.29336, 42.89416, 42.94616, 43.23666, 43.39517, 43.57509, 43.54637, 43.90628, 43.89597, 43.97261, 44.12801, 43.89708, 43.8616, 43.9502, 43.9412, 44.02615, 43.94618, 43.93134, 43.98629, 43.84266, 44.1197, 43.80469, 43.8161, 43.86501, 43.78159, 43.55139, 43.52779, 43.54522, 43.51697, 43.21266, 42.93177, 42.50974, 42.39488, 42.18402, 42.18699, 42.42397, 41.7017, 41.246, 39.19218, 38.49695, 38.1042, 35.95146]
wrk_hours_CPS_nonc_black = [22.83747, 29.41613, 34.29444, 36.3078, 37.92713, 39.64614, 38.95276, 39.50281, 40.2061, 40.23299,40.52941,40.17025,40.58729,40.77788,40.43398,40.70308,41.10214,40.34703,40.68182,40.73382,39.96592,41.11389,40.98627,41.03655,40.78833,40.89331,41.17895,41.41185,41.23407,40.51578,40.80245,40.92878,40.68591,40.89167,41.44828,40.97869,40.88929,40.7767,40.8551,39.93467,40.11656,39.61204,38.77016,38.69668,40.54819,38.71429,37.40426,34.41509, 38.91667]
ages_CPS_nonc = collect(17:65)
ages_CPS_c = collect(19:65)

47-element Array{Int64,1}:
 19
 20
 21
 22
 23
 24
 25
 26
 27
 28
 29
 30
 31
  ⋮
 54
 55
 56
 57
 58
 59
 60
 61
 62
 63
 64
 65

In [29]:
combined_all = [collect(wrk_hours_NLSY_all[1:21]);collect(wrk_hours_CPS_all[25:end])]
combined_nonc = [collect(wrk_hours_NLSY_nonc[1:21]);collect(wrk_hours_CPS_nonc[25:end])]
combined_nonc_black = [collect(wrk_hours_NLSY_nonc_black[1:21]);collect(wrk_hours_CPS_nonc_black[25:end])]
combined_c = [collect(wrk_hours_NLSY_c[1:21]);collect(wrk_hours_CPS_c[23:end])]
combined_ages = collect(20:65)

46-element Array{Int64,1}:
 20
 21
 22
 23
 24
 25
 26
 27
 28
 29
 30
 31
 32
  ⋮
 54
 55
 56
 57
 58
 59
 60
 61
 62
 63
 64
 65

In [30]:
plot(combined_ages, combined_c, label="College")
plot!(combined_ages, combined_nonc,label="Non-College" )

ErrorException: error compiling _plot!: error compiling _display: could not load library "libGR.so"
dlopen(libGR.so.dylib, 1): image not found

In [31]:
median_first_stage

288.90108179706164